# NIS800-53 analysis based on the new NLP-based Related Controls

This notebook is the evolution of the [NIS800-53 analysis](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb) notebook and uses the results from the NLP analysis that was done in the [NIS800-53 analysis NLP](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis%20NLP.ipynb) notebook and utilizes the functions written in the [helping_functions](https://github.com/lior0110/regulations_analysis/blob/main/helping_functions.py) file.

The goal of this notebook is to explore how different relationship measures affect the group clustering of the NIS800-53 cybersecurity regulations.
We will take the new relationships measures we have made in the [NIS800-53 analysis NLP](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis%20NLP.ipynb) notebook and do similar analyses to what we have done in the [NIS800-53 analysis notebook](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb) using the functions from the [helping_functions](https://github.com/lior0110/regulations_analysis/blob/main/helping_functions.py) file, we will check and see what the effects of the new relationships measures on the group clustering and discuss if the new relationships measures are a good human-independent relationships measures.

The *"relationship level/threshold"* was chosen to create a comparable number of total connections to human-made relationships.

In [2]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import networkx as nx

import google.generativeai as genai
from google.api_core import retry

import os
import time

# import all the helping functions that were written in the base NIS800-53 analysis notebook
from helping_functions import *


# Read the NIS800-53 regulations with the new NLP-based related controls

In [3]:

# NIS_regulations = pd.read_excel("regulations//sp800-53r5-control-catalog.xlsx")
# NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls.feather")
# NIS_regulations = pd.read_parquet("regulations//Active_NIS_regulations_with_new_related_controls.parquet")
# # Constructed so any of the new "related controls" columns will give a similar number of connections to the default "related controls"
# NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls2L.feather")
# NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls2.feather")
# Constructed so the combination of the new "related controls" columns will give a similar number of connections to the default "related controls"
NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls2M.feather")
NIS_regulations

# remove every key with None value for the new relations columns and sort them from highest to lowest

NIS_regulations.loc[:, "bm25 Related Controls"] = NIS_regulations.loc[:, "bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))
NIS_regulations.loc[:, "normalized bm25 Related Controls"] = NIS_regulations.loc[:, "normalized bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))

NIS_regulations.loc[:, "SentenceTransformer Related Controls"] = NIS_regulations.loc[:, "SentenceTransformer Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))

NIS_regulations.loc[:, "SaaS Related Controls"] = NIS_regulations.loc[:, "SaaS Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))
# NIS_regulations.loc[:, "bm25 Related Controls"] = NIS_regulations.loc[:, "bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "normalized bm25 Related Controls"] = NIS_regulations.loc[:, "normalized bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "SentenceTransformer Related Controls"] = NIS_regulations.loc[:, "SentenceTransformer Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "SaaS Related Controls"] = NIS_regulations.loc[:, "SaaS Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})

NIS_regulations


,Control Identifier,Control (or Control Enhancement) Name,Control Text,Discussion,Related Controls,Main Control Name,Family Name,Related Controls Dictionary,Full Text,bm25 Related Controls,normalized bm25 Related Controls,SentenceTransformer Related Controls,SaaS Related Controls
0,AC-1,Policy and Procedures,"a. Develop, document, and disseminate to [Assi...",Access control policy and procedures address t...,"IA-1, PM-9, PM-24, PS-8, SI-12 .",AC-1,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...","Policy and Procedures\n\na. Develop, document,...","{'MA-1': 59.27978515625, 'PL-1': 59.2797851562...","{'MA-1': 0.8947177027189795, 'PL-1': 0.8947177...","{'PS-1': 0.9554650783538818, 'CA-1': 0.9547804...","{'CA-1': 0.9555830106820367, 'SC-1': 0.9506337..."
1,AC-2,Account Management,a. Define and document the types of accounts a...,Examples of system account types include indiv...,"AC-3, AC-5, AC-6, AC-17, AC-18, AC-20, AC-24, ...",AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management\n\na. Define and document t...,"{'AC-2(3)': 63.61791229248047, 'AC-2(7)': 62.1...","{'AC-2(3)': 0.5456564097423026, 'AC-2(7)': 0.5...","{'AC-2(7)': 0.8722906112670898, 'AC-2(11)': 0....","{'AC-2(7)': 0.8690222363681739, 'AC-2(11)': 0...."
2,AC-2(1),Account Management | Automated System Account ...,Support the management of system accounts usin...,Automated system account management includes u...,None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Automated System Account ...,{},"{'AC-2(2)': 0.6856685718365872, 'AC-2(8)': 0.6...","{'AC-2(8)': 0.8399572968482971, 'IR-4(1)': 0.8...","{'AC-2(8)': 0.8385143606316967, 'AC-2(11)': 0...."
3,AC-2(2),Account Management | Automated Temporary and E...,Automatically [Selection: remove; disable] tem...,Management of temporary and emergency accounts...,None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Automated Temporary and E...,{},"{'AC-2(4)': 0.4500737751259309, 'AC-2(3)': 0.4...","{'AC-2(3)': 0.8158748745918274, 'AC-2(4)': 0.8...","{'AC-2(3)': 0.8327054165938192, 'AC-2(13)': 0...."
4,AC-2(3),Account Management | Disable Accounts,Disable accounts within [Assignment: organizat...,"Disabling expired, inactive, or otherwise anom...",None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Disable Accounts\n\nDisab...,{'AC-2(13)': 23.608457565307617},"{'AC-2(13)': 0.5667917968522473, 'AC-2(2)': 0....","{'AC-2(13)': 0.8616950511932373, 'AC-2(5)': 0....","{'AC-2(13)': 0.8635755082149525, 'AC-2(2)': 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,SR-11,Component Authenticity,a. Develop and implement anti-counterfeit poli...,Sources of counterfeit components include manu...,"PE-3, SA-4, SI-7, SR-9, SR-10.",SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity\n\na. Develop and imple...,"{'SR-11(1)': 28.72646141052246, 'SR-11(3)': 26...","{'SR-11(1)': 0.6496216599515963, 'SR-11(3)': 0...","{'SR-11(1)': 0.8409212231636047, 'SR-11(3)': 0...","{'SR-11(1)': 0.8345096504298886, 'CM-8(3)': 0...."
1185,SR-11(1),Component Authenticity | Anti-counterfeit Trai...,Train [Assignment: organization-defined person...,None.,AT-3.,SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity | Anti-counterfeit Trai...,{},"{'SR-11': 0.6172898806384338, 'SR-11(3)': 0.54...","{'SR-11(3)': 0.8443235158920288, 'SR-11': 0.84...","{'SR-11(3)': 0.8393997564526279, 'SR-11': 0.83..."
1186,SR-11(2),Component Authenticity | Configuration Control...,Maintain configuration control over the follow...,None.,"CM-3, MA-2, MA-4, SA-10.",SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity | Configuration Control...,{},{'SA-4(5)': 0.37485503792576363},"{'MA-2': 0.8005739450454712, 'CM-8(2)': 0.7839...","{'SR-4': 0.8314977910120207, 'MA-2': 0.8313986..."
1187,SR-11(3),Component Authenticity | Anti-counterfeit Scan...,Scan for cou

In [4]:

family_column = "Family Name"

# group by family name and count the number of regulations in each family
NIS_regulations.groupby(by=[family_column]).apply(len,include_groups=False).mean()
# now interquntile



50.35

In [5]:

NIS_regulations.loc[1,"normalized bm25 Related Controls"]
# NIS_regulations.loc[1,["bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SaaS Related Controls"]]
# NIS_regulations.loc[0,"bm25 Related Controls"].keys()
# type(NIS_regulations.loc[0,"bm25 Related Controls"])

# print all key value pairs in NIS_regulations.loc[1,"normalized bm25 Related Controls"]
for k, v in NIS_regulations.loc[1,"normalized bm25 Related Controls"].items():
    print(f"{k}: {v}")
    print("-"*80)
    




AC-2(3): 0.5456564097423026
--------------------------------------------------------------------------------
AC-2(7): 0.5332102684763498
--------------------------------------------------------------------------------
AC-2(9): 0.5220074162177839
--------------------------------------------------------------------------------
AC-2(12): 0.5073050386463656
--------------------------------------------------------------------------------
AC-2(2): 0.497907489534503
--------------------------------------------------------------------------------
AC-2(13): 0.4820235309761175
--------------------------------------------------------------------------------
IA-2(5): 0.4703392514575003
--------------------------------------------------------------------------------
IA-5: 0.46017980787930934
--------------------------------------------------------------------------------
AC-2(8): 0.4444947807971446
--------------------------------------------------------------------------------
AC-6(2): 0.441447301

## transform the old Related Controls from the string NIS representation to the new Dictionary representation

In [6]:

# Related_Controls = row["Related Controls"].replace(" .","").replace(".","").split(", ")
# apply the replace and split operation on the Related Controls column to get a list of related controls
# NIS_regulations["Related Controls"].apply(lambda x: {k: v for k, v in (i.split("-") for i in x.replace(" .","").replace(".","").split(", ")) if v is not None else None })
new_NIS_Related_Controls = NIS_regulations["Related Controls"].apply(lambda x: x.replace(" .","").replace(".","").split(", "))
# transform the list of related controls into a dictionary where the key is the control and the value is 0.5
# new_NIS_Related_Controls[3]
# {x: 0.5 for x in new_NIS_Related_Controls[0] if x != "None"}
new_NIS_Related_Controls = new_NIS_Related_Controls.apply(lambda x: {k: 0.5 for k in x if k!= "None"})
NIS_regulations["Related Controls Dictionary"] = new_NIS_Related_Controls

new_NIS_Related_Controls


0       {'IA-1': 0.5, 'PM-9': 0.5, 'PM-24': 0.5, 'PS-8...
1       {'AC-3': 0.5, 'AC-5': 0.5, 'AC-6': 0.5, 'AC-17...
2                                                      {}
3                                                      {}
4                                                      {}
                              ...                        
1184    {'PE-3': 0.5, 'SA-4': 0.5, 'SI-7': 0.5, 'SR-9'...
1185                                        {'AT-3': 0.5}
1186    {'CM-3': 0.5, 'MA-2': 0.5, 'MA-4': 0.5, 'SA-10...
1187                                        {'RA-5': 0.5}
1188                                        {'MP-6': 0.5}
Name: Related Controls, Length: 1007, dtype: object

# Checking the "Related Controls" column

We want to know what the statistics are of all of the Related Controls columns (the original column and the new columns we have made).

## Run describe on the "Related Controls" column

In [7]:
NIS_regulations.loc[:,["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SaaS Related Controls"]].describe(include="all")

,Related Controls Dictionary,bm25 Related Controls,normalized bm25 Related Controls,SentenceTransformer Related Controls,SaaS Related Controls
count,1007,1007,1007,1007,1007
unique,540,442,873,919,888
top,{},{},{},{},{}
freq,361,566,135,89,120


We can see that like in the human-made relations there are still some regulations that have no related controls.

## Look if the controls are mutually related
Two controls will be considered mutually related if control A or any of its sub-controls is related to control B or any of its sub-controls and control B or any of its sub-controls are also related to control A or any of its sub-controls.

We assume that related controls should be mutually related because there is no reason that if control A is related to control B control B will not be related to control A.

In order to check the mutual relations status and get relations statistics we will use the **'get_mutual_relations'** function from the **helping_functions**



In [8]:

relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SaaS Related Controls"]

# get the statistics for every individual relationship method
for relation_column in relations_columns:
    mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,relation_column,do_print=False)
    print(f'there are {len(not_mutually_related)} non-mutual connections in "{relation_column}"')
    print(f"in-family connections:\n{in_family_connection_statistics}")
    # print(f"Relations statistics:\n{relations_statistics}")
    s_r = sum(mutually_related["same family"]) / len(mutually_related)
    if len(not_mutually_related) > 0:
        s_r_n = sum(not_mutually_related["same family"]) / len(not_mutually_related)
    else:
        s_r_n = 0
    s = (sum(mutually_related["same family"]) + sum(not_mutually_related["same family"])) / (len(mutually_related) + len(not_mutually_related))
    print(f"{s:.2%} of existing connections are in-family")
    print(f"{s_r:.2%} of existing mutual connections are in-family")
    print(f"{s_r_n:.2%} of existing non-mutual connections are in-family")
    print("-"*100)
    

# get the statistics for a combination of the three human-independent relationship method (excluding the un-normalized bm25 Related Controls)
mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,relations_columns[2:],do_print=False)

print(f'there are {len(not_mutually_related)} non-mutual connections in "{relations_columns[2:]}"')
print(f"in-family connections:\n{in_family_connection_statistics}")
# print(f"Relations statistics:\n{relations_statistics}")
s_r = sum(mutually_related["same family"]) / len(mutually_related)
if len(not_mutually_related) > 0:
    s_r_n = sum(not_mutually_related["same family"]) / len(not_mutually_related)
else:
    s_r_n = 0
s = (sum(mutually_related["same family"]) + sum(not_mutually_related["same family"])) / (len(mutually_related) + len(not_mutually_related))
print(f"{s:.2%} of existing connections are in-family")
print(f"{s_r:.2%} of existing mutual connections are in-family")
print(f"{s_r_n:.2%} of existing non-mutual connections are in-family")
print("-"*100)



there are 645 non-mutual connections in "Related Controls Dictionary"
in-family connections:
number of families                                                                                      20.000000
mean number of controls in family                                                                       14.900000
median number of controls in family                                                                     12.000000
std of number of controls in family                                                                     10.187195
Inter-quartile range of number of controls in family                                                     9.500000
number of participating controls                                                                       298.000000
balance score 1 (mean/std)                                                                               1.462620
balance score 2 (median/Inter-quartile range)                                                            1.26

For the related controls we got from "bm25 Related Controls", we see a lot of non-mutual connections, 1567 compared to 645 from the human-made relations. This may be because the BM25 score function is not commutative so the distance from control A to control B is not the same as from control B to control A. Apart from that, the non-normalized bm25 scores look different from all the other tested methods, and it gives worse statistical indicators than them and the human-made relations.

For the related controls we got from "normalized bm25 Related Controls", we see some non-mutual connections, 1014 compared to 645 from the human-made relations. This is probably due to the score normalization step we used.

For the related controls we got from "SentenceTransformer Related Controls" and "SaaS Related Controls" we got 0 non-mutual connections. This is because the scores from them are normalized by default and because they are commutative so the distance from control A to control B is the same as from control B to control A.

Furthermore (Apart from the "bm25 Related Controls" scores), the statistics of the in-family connections are all on similar levels to what we got from the human-made relations for the three individual human-independent relationship method. 

The combination of the three individual human-independent relationship methods, "normalized bm25 Related Controls, "SentenceTransformer Related Controls," and "SaaS Related Controls," gives a similar number of non-mutual connections to the human-made connections, 675 compared to 645.
The combined result also shows more than 60% more connections than the individual methods, saying that the connections from the three individual human-independent relationship methods do not completely overlap but still have some base similarities.

The statistical indicators of the combined result are lower than the human-made baseline, which may be expected because of the results we saw in the first analysis [NIS800-53 analysis](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb).
But combined with the statistical indicators of the individual human-independent relationship method being similar to the human-made baseline, may say that each one of the three human-independent relationship methods see small aspects that the other two methods don't see.

# Make a network graph based on the "Related Controls" column/s


## Recreate the results from the previous NIS800-53 analysis

We will start by using the original "Related Controls" column in the new pipeline with the new functions in order to see that we get similar results to what we got in the first analysis [NIS800-53 analysis](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb).

In [9]:

relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SaaS Related Controls"]

is_directed:bool = False
is_Main_Controls_Only:bool = True
is_Weighted:bool = True

add_groups_summaries:bool = False

i:int=1
Relations = relations_columns[0] # use the original "Related Controls" column

Related_Controls_Graph = make_network_graph_from_relations(NIS_regulations, Relations, 
                                                           is_directed=is_directed, is_Main_Controls_Only=is_Main_Controls_Only, is_Weighted=is_Weighted)
if is_directed:
    strongly_connected_components, weakly_connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(weakly_connected_components[0]),
                                                                                  NIS_regulations,Relations,Is_Weighted=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=True, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

else:
    connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(connected_components[0]),
                                                                                  NIS_regulations,Relations,Is_Weighted=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=True, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

    in_family_connection_statistics_test





number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 1906
number of connected components: 6
component 0 has 293 nodes
component 1 has 1 nodes
{'AT-6'}
component 2 has 1 nodes
{'MP-8'}
component 3 has 1 nodes
{'SC-51'}
component 4 has 1 nodes
{'SI-22'}
component 5 has 1 nodes
{'SI-23'}
----------------------------------------------------------------------------------------------------
base NIS800-53 families:
the in_family_connection_ratios are: 0.248 , 0.232
number of families                                                                                      20.000000
mean number of controls in family                                                                       14.900000
median number of controls in family                                                                     12.000000
std of number of controls in family                                         

In [10]:


with pd.ExcelWriter('regulations outputs\\in family connection statistics test old.xlsx') as writer:  
    in_family_connection_statistics_test.to_excel(writer, sheet_name="methods comparison",merge_cells=False,freeze_panes=(1,1))
    for i in range(len(relations_DataFrames)) :
        relations_DataFrames[i].to_excel(writer, sheet_name=relations_DataFrames[i].index.name, merge_cells=False, freeze_panes=(1,1))
    # NIS_relations_statistics.to_excel(writer, sheet_name="original NIS statistics",merge_cells=False,freeze_panes=(1,1))
    # Modularity_based_communities_relations_statistics.to_excel(writer, sheet_name="Modularity statistics",merge_cells=False,freeze_panes=(1,1))
    # Louvain_Communities_relations_statistics.to_excel(writer, sheet_name="Louvain statistics",merge_cells=False,freeze_panes=(1,1))
    # Fluid_Communities_relations_statistics.to_excel(writer, sheet_name="Fluid statistics",merge_cells=False,freeze_panes=(1,1))

in_family_connection_statistics_test


,NIS800-53 families,Modularity based communities,Louvain Communities,Fluid Communities,Divisive Communities,Label propagation,Centrality Communities
number of families,20.000000,7.000000,7.000000,20.000000,20.000000,2.000000,20.000000
mean number of controls in family,14.900000,41.857143,41.857143,14.650000,14.650000,146.500000,14.650000
median number of controls in family,12.000000,51.000000,40.000000,15.000000,2.000000,146.500000,1.000000
std of number of controls in family,10.187195,33.338809,15.323496,3.731445,50.239715,204.353860,59.869397
Inter-quartile range of number of controls in family,9.500000,63.000000,25.500000,5.500000,3.000000,144.500000,1.000000
number of participating controls,298.000000,293.000000,293.000000,293.000000,293.000000,293.000000,293.000000
balance score 1 (mean/std),1.462620,1.255508,2.731566,3.926093,0.291602,0.716894,0.244699
balance score 2 (median/Inter-quartile range),1.263158,0.809524,1.568627,2.727273,0.666667,1.013841,1.000000
same family connections,785.000000,1812.000000,1618.000000,1125.000000,2872.000000,3165.000000,3109.000000
out of family connections,2382.000000,1355.000000,1549.000000,2042.000000,295.000000,2.000000,58.000000


In [11]:


relations_DataFrames[0].index.name

families = NIS_regulations.groupby(by=["Family Name"]) # pre-group families
# get number of distinct controls in the family
families['Main Control Name'].nunique()
num_Controls_in_family = families.apply(len,include_groups=False)

num_Controls_in_family


Family Name
AC    131
AT     15
AU     56
CA     25
CM     56
CP     49
IA     55
IR     40
MA     28
MP     20
PE     51
PL     11
PM     37
PS     17
PT     21
RA     22
SA    106
SC    139
SI    101
SR     27
dtype: int64

Similar to conclusions from the first analysis [NIS800-53 analysis](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb) ***Communities conclusion*** sub-section:
* "Divisive Communities" "Label propagation" and "Centrality Communities" gives us very unbalanced new families that we don't want 
* "Modularity based communities" gives exactly the same results, but now we can see that the new families may be not balanced like we would have wanted
* "Louvain Communities" having a random element gives similar but not exactly the same results, having better **empiric connection probabilities** scores then the default families and always having better score at **statistical connection probability 1 (Over-representation of total inner connections)** and having close **statistical connection probability 2 (Over-representation of mean inner connections)** and **statistical connection probability 3 (mean of Over-representation of inner connections per family)** scores to the default families (sometimes a little more sometimes a little less), but we can also see that its balance scores are better than the default families.
* "Fluid Communities" having a random element gives similar but not exactly the same results, but like in the previous analysis gives better **empiric connection probabilities** and **statistical connection probabilities** scores than the default families, and we can also see that its balance scores are better than the default families.


After seeing that the pipeline works ok by recreating the original analysis results, we can move on to run the pipeline on the new NLP based "Related Controls" columns we have created.

## Calculate Clustering from new NLP-based "Related Controls" columns

Now we will choose the subset of the NLP-based "Related Controls" columns we have created and run the pipeline based on them.

In [12]:

relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SaaS Related Controls"]

is_directed:bool = False
is_Main_Controls_Only:bool = True
is_Weighted:bool = True

add_groups_summaries:bool = False

i:int=1
Relations = relations_columns[2:] # use the original "Related Controls" column

Related_Controls_Graph = make_network_graph_from_relations(NIS_regulations, Relations, 
                                                           is_directed=is_directed, is_Main_Controls_Only=is_Main_Controls_Only, is_Weighted=is_Weighted)
if is_directed:
    strongly_connected_components, weakly_connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(weakly_connected_components[0]),
                                                                                  NIS_regulations,Relations,Is_Weighted=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=True, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

else:
    connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(connected_components[0]),
                                                                                  NIS_regulations,Relations,Is_Weighted=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=True, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

    in_family_connection_statistics_test





number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 1856
number of connected components: 13
component 0 has 283 nodes
component 1 has 3 nodes
{'SC-22', 'SC-21', 'SC-20'}
component 2 has 2 nodes
{'PM-24', 'PT-8'}
component 3 has 1 nodes
{'PM-13'}
component 4 has 1 nodes
{'PM-23'}
component 5 has 1 nodes
{'PM-26'}
component 6 has 1 nodes
{'RA-6'}
component 7 has 1 nodes
{'SC-6'}
component 8 has 1 nodes
{'SC-26'}
component 9 has 1 nodes
{'SC-27'}
component 10 has 1 nodes
{'SC-31'}
component 11 has 1 nodes
{'SC-44'}
component 12 has 1 nodes
{'SI-11'}
----------------------------------------------------------------------------------------------------
base NIS800-53 families:
the in_family_connection_ratios are: 0.259 , 0.25
number of families                                                                                      20.000000
mean number of controls in family    

In [13]:


with pd.ExcelWriter('regulations outputs\\in family connection statistics test new NLP based.xlsx') as writer:  
    in_family_connection_statistics_test.to_excel(writer, sheet_name="methods comparison",merge_cells=False,freeze_panes=(1,1))
    for i in range(len(relations_DataFrames)) :
        relations_DataFrames[i].to_excel(writer, sheet_name=relations_DataFrames[i].index.name, merge_cells=False, freeze_panes=(1,1))
    # NIS_relations_statistics.to_excel(writer, sheet_name="original NIS statistics",merge_cells=False,freeze_panes=(1,1))
    # Modularity_based_communities_relations_statistics.to_excel(writer, sheet_name="Modularity statistics",merge_cells=False,freeze_panes=(1,1))
    # Louvain_Communities_relations_statistics.to_excel(writer, sheet_name="Louvain statistics",merge_cells=False,freeze_panes=(1,1))
    # Fluid_Communities_relations_statistics.to_excel(writer, sheet_name="Fluid statistics",merge_cells=False,freeze_panes=(1,1))

in_family_connection_statistics_test


,NIS800-53 families,Modularity based communities,Louvain Communities,Fluid Communities,Divisive Communities,Label propagation,Centrality Communities
number of families,20.000000,11.000000,10.000000,20.000000,20.000000,6.000000,20.000000
mean number of controls in family,14.900000,25.727273,28.300000,14.150000,14.150000,47.166667,14.150000
median number of controls in family,12.000000,24.000000,25.000000,13.500000,7.000000,2.000000,1.000000
std of number of controls in family,10.187195,21.028984,14.158233,5.018649,17.070365,110.146115,57.164836
Inter-quartile range of number of controls in family,9.500000,28.500000,16.500000,4.500000,13.250000,0.750000,1.000000
number of participating controls,298.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
balance score 1 (mean/std),1.462620,1.223420,1.998837,2.819484,0.828922,0.428219,0.247530
balance score 2 (median/Inter-quartile range),1.263158,0.842105,1.515152,3.000000,0.528302,2.666667,1.000000
same family connections,856.000000,1458.000000,1179.000000,1517.000000,1774.000000,3265.000000,3263.000000
out of family connections,2454.000000,1852.000000,2131.000000,1793.000000,1536.000000,45.000000,47.000000


Similar to the original case with the human-made "Related Controls" column "Divisive Communities" "Label propagation" and "Centrality Communities" gives us very unbalanced new families that we don't want.

"Modularity based communities" once again give us lower balance scores than we would like (and are lower than the default families).

Both the "Modularity based communities" and the "Louvain Communities" give us higher **empiric connection probabilities** scores than the default families, but their **statistical connection probabilities** scores are lower than the default families. 

"Fluid Communities" is the best performer here, giving better **balance scores** than the default families, and both its **empiric connection probabilities** scores and **statistical connection probabilities** scores are better than the default families.



After saying all of that, we still need to also say that none of the methods has recreated the human-made family splits in any significant way, indicating one more time that the human-made families may raise more out of the human mind than from the connections between the different regulations.




# Final Conclusions

We have started by making a pipeline using the analysis done in the [NIS800-53 analysis NLP](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis%20NLP.ipynb) notebook and extended it with functionality from the [helping_functions](https://github.com/lior0110/regulations_analysis/blob/main/helping_functions.py) file to make a more automated pipeline similar to what we done in the original [NIS800-53 analysis](https://github.com/lior0110/regulations_analysis/blob/main/NIS800-53%20analysis.ipynb) notebook that is now more automated and mostly human-independent (completing the future works from last time).

We have once again shown that we can not recreate the original families from the connection data between the regulations in any significant way, implying that the **NIS800-53** families are at least not optimal.

We have been surprised to see that using the ***human-independent connections data*** we got better results on the original human-made families than what we got from using the ***human-made connections data***(?!).
But still, we have shown that we have at least one method ("Fluid Communities") that we can use that still outperforms the original families in every one of our metrics, implying that the **NIS800-53** families are still sub-optimal and can be improved.







# Future Work

In this study, we utilized existing clustering algorithms, which performed well and provided better results than the default options. However, these algorithms currently group each regulation into a single family. For future work, we should consider relaxing this restriction, allowing each regulation to be associated with multiple families. This approach would be more logical, as it recognizes that a single regulation can contribute to various topics and, therefore, belong to more than one family.



# Test codes

In [14]:

len(relations_DataFrames)
relations_DataFrames[2]
# relations_DataFrames[2].loc[:,["actual inside relations","actual outside relations"]]
# NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ])
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ]),:]
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SA-9" ]),:]
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ]),relations_columns[i]]
# [SC-20, SC-21, SC-22]
# connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=True)
# Related_Controls_Graph["SC-22"]
# Related_Controls_Graph["SA-9"]
# Figure = plt.figure(figsize=(30,30))
# nx.draw_kamada_kawai(Related_Controls_Graph, with_labels=True, font_weight='bold')
# nx.draw_spring(Related_Controls_Graph, with_labels=True, font_weight='bold', edge_color='purple')
i=0
relations_DataFrames[1].loc[i,"names of regulations"]
group = NIS_regulations["Main Control Name"].isin(relations_DataFrames[1].loc[i,"names of regulations"])
NIS_regulations.loc[group]
# NIS_regulations.loc[group,"Full Text"]

# print(NIS_regulations.loc[group,"Full Text"])
regulations_examples:str = ""
for i,regulations_text in enumerate(NIS_regulations.loc[group,"Full Text"]):
    regulations_examples += f"Regulation {i+1}:\n{regulations_text}\n\n"

    # break
print(regulations_examples)


Regulation 1:
Policy and Procedures

a. Develop, document, and disseminate to [Assignment: organization-defined personnel or roles]:
1. [Selection (one or more): Organization-level; Mission/business process-level; System-level] access control policy that:
(a) Addresses purpose, scope, roles, responsibilities, management commitment, coordination among organizational entities, and compliance; and
(b) Is consistent with applicable laws, executive orders, directives, regulations, policies, standards, and guidelines; and
2. Procedures to facilitate the implementation of the access control policy and the associated access controls;
b. Designate an [Assignment: organization-defined official] to manage the development, documentation, and dissemination of the access control policy and procedures; and
c. Review and update the current access control:
1. Policy [Assignment: organization-defined frequency] and following [Assignment: organization-defined events]; and
2. Procedures [Assignment: organ

In [15]:

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

for model in genai.list_models():
  # print(model.name)
  # print(model.supported_generation_methods)
  if 'generateContent' in model.supported_generation_methods:
    print(model.name)



models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/gemma-3-27b-i

In [16]:

# genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
# system_instruction:str = "You are an expert in cyber regulations. \
#   Your task is to read all the regulations you are given and summarize them in up to 5 short and concise bullet points. \
#   In your answers give the general topic that represents all of the regulations you were given."
# model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp",system_instruction=system_instruction)
# retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# response = model.generate_content(f"Please summarize this group of cyber regulations:\n\n{regulations_examples}",
#             request_options=retry_policy,)

# response




In [17]:

# print(response.text)


In [18]:

# # add summarizion to the groups of regulations
# def add_summation(regulations_DataFrame:pd.DataFrame, relations_DataFrame:pd.DataFrame, is_Main_Controls_Only:bool=True) -> pd.DataFrame:

#     for inx in relations_DataFrame.index:
#         # get the relevant regulations
#         if is_Main_Controls_Only:
#             group = regulations_DataFrame["Main Control Name"].isin(relations_DataFrame.loc[inx,"names of regulations"])
#         else:
#             group = regulations_DataFrame["Control Identifier"].isin(relations_DataFrame.loc[inx,"names of regulations"])
#         regulations_examples:str = ""
#         for i,regulations_text in enumerate(NIS_regulations.loc[group,"Full Text"]):
#             regulations_examples += f"Regulation {i+1}:\n{regulations_text}\n\n"

#             # break
#         # print(regulations_examples)

#         # summarize the regulations
#         system_instruction:str = "You are an expert in cyber regulations. \
#         Your task is to read all the regulations you are given and summarize them in up to 5 short and concise bullet points. \
#         In your answers give the general topic that represents all of the regulations you were given."
#         model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp",system_instruction=system_instruction)
#         retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}
#         response = model.generate_content(f"Please summarize this group of cyber regulations:\n\n{regulations_examples}",
#                                           request_options=retry_policy,)
        
#         # add the responses to the summary column in the relations_DataFrame
#         relations_DataFrame.loc[inx,"Summary"] = response.text.strip()

#         time.sleep(6) # sleep for a little bit because of RPM limitations

#     return relations_DataFrame

# relations_DataFrame_with_summary = add_summation(NIS_regulations,relations_DataFrames[1],is_Main_Controls_Only=True)
# relations_DataFrame_with_summary


In [19]:
# # print all the summaries
# for inx in relations_DataFrame_with_summary.index:
#     print(f"Regulation Group {inx+1}:\nSummary: {relations_DataFrame_with_summary.loc[inx,'Summary']}\n\n")
#     print("-"*100)
#     # break




In [20]:

in_family_connection_statistics_test
# relations_DataFrames



,NIS800-53 families,Modularity based communities,Louvain Communities,Fluid Communities,Divisive Communities,Label propagation,Centrality Communities
number of families,20.000000,11.000000,10.000000,20.000000,20.000000,6.000000,20.000000
mean number of controls in family,14.900000,25.727273,28.300000,14.150000,14.150000,47.166667,14.150000
median number of controls in family,12.000000,24.000000,25.000000,13.500000,7.000000,2.000000,1.000000
std of number of controls in family,10.187195,21.028984,14.158233,5.018649,17.070365,110.146115,57.164836
Inter-quartile range of number of controls in family,9.500000,28.500000,16.500000,4.500000,13.250000,0.750000,1.000000
number of participating controls,298.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
balance score 1 (mean/std),1.462620,1.223420,1.998837,2.819484,0.828922,0.428219,0.247530
balance score 2 (median/Inter-quartile range),1.263158,0.842105,1.515152,3.000000,0.528302,2.666667,1.000000
same family connections,856.000000,1458.000000,1179.000000,1517.000000,1774.000000,3265.000000,3263.000000
out of family connections,2454.000000,1852.000000,2131.000000,1793.000000,1536.000000,45.000000,47.000000
